In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data=pd.read_csv(r"C:\Users\anilm\Downloads\WELFake_Dataset.csv\WELFake_Dataset.csv")

In [ ]:
print(data.shape)

In [ ]:
data.head(1)

In [ ]:
print(data.isnull().sum())

In [ ]:
data=data.fillna(' ')

In [ ]:
print(data.isnull().sum())

In [ ]:
data.head()

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
# Download necessary NLTK datasets
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Keep only alphanumeric and spaces
    return text

In [ ]:
    sentences = sent_tokenize(text)
    tokens = [word_tokenize(sentence) for sentence in sentences]
    filtered_tokens = [[word for word in token_list if word not in stop_words] for token_list in tokens]
    return filtered_tokens

In [ ]:
def regex_filter(text):
    """Remove numbers, URLs, emails, and phone numbers."""
    text = re.sub(r'\d+', '', text) 
    text = re.sub(r'http\S+|www\S+', '', text)  
    text = re.sub(r'\S+@\S+', '', text) 
    text = re.sub(r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', '', text) 
    return text

In [ ]:
import re
def fin(text):
  text=  re.sub(r'[^\w\s]', ' ',text)
  text = re.sub(r'[ \n]+', ' ', text)
  return text

In [ ]:
def lemmatize_text(tokens):
    """Lemmatize tokens."""
    lemmatized_tokens = [[lemmatizer.lemmatize(word) for word in token_list] for token_list in tokens]
    return lemmatized_tokens

In [ ]:
def full_preprocess_pipeline(text):
    text = preprocess_text(text)  # Lowercase and remove non-essential characters
    text = regex_filter(text)  # Remove numbers, URLs, emails, and phone numbers
    text = fin(text)  # Remove special characters and unwanted spaces
    tokens = tokenize_and_remove_stopwords(text)  
    lemmatized_tokens = lemmatize_text(tokens)  
    cleaned_text = ' '.join([' '.join(sentence) for sentence in lemmatized_tokens])
    return cleaned_text

In [ ]:


# Apply preprocessing to the 'title' and 'text' columns using the full preprocessing pipeline
data['cleaned_text'] = data['text'].apply(full_preprocess_pipeline)
data['cleaned_title'] = data['title'].apply(full_preprocess_pipeline)

# Optionally, drop the 'Unnamed: 0' column if it's not needed
if 'Unnamed: 0' in data.columns:
    data = data.drop(columns=['Unnamed: 0'])

print(data.head())

# Save the processed DataFrame to a new CSV file
data.to_csv('processeddata.csv', index=False)


In [ ]:
processed_data=pd.read_csv(r"C:\Users\anilm\Downloads\processeddata.csv")

In [ ]:
print(processed_data.head())

In [ ]:
import fasttext
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv(r"C:\Users\anilm\Downloads\processeddata.csv")

# Combine 'cleaned_title' and 'cleaned_text' columns for FastText input
df['combined_text'] = df['cleaned_title'].fillna('') + ' ' + df['cleaned_text'].fillna('')

# Format labels for FastText (__label__0 for real news, __label__1 for fake news)
df['fasttext_format'] = '__label__' + df['label'].astype(str) + ' ' + df['combined_text']

train_data, test_data = train_test_split(df['fasttext_format'], test_size=0.2, random_state=42)

# Save the formatted data into text files for FastText
with open('fasttext_train.txt', 'w', encoding='utf-8') as train_file:
    train_file.write('\n'.join(train_data))

with open('fasttext_test.txt', 'w', encoding='utf-8') as test_file:
    test_file.write('\n'.join(test_data))


In [ ]:

# Train a supervised FastText model with specified parameters
model = fasttext.train_supervised(
    input="fasttext_train.txt",
    lr=0.01,  
    epoch=50,  
    wordNgrams=3,  # Use word n-grams of size 3
    dim=300  #  word vectors dimension
)

# Saving the trained model
model.save_model("supervised_fasttext_model.bin")

# Evaluate the model on the test data
result = model.test("fasttext_test.txt")

# Print evaluation results
print("Number of samples:", result[0])
print("Precision:", result[1])
print("Recall:", result[2])

# Predicting new examples (optional)
examples = [
    "Breaking news: Market crashes after economic policy announcement.",
    "NASA discovers signs of alien life on Mars!"
]
predictions = model.predict(examples)

# Display predictions
for example, prediction in zip(examples, predictions[0]):
    print(f"Text: {example}")
    print(f"Prediction: {prediction}")


In [ ]:
import fasttext
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

In [ ]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel

# Tokenize the processed texts (each document is a list of words)
processed_data1 = [text.split() for text in df['combined_text']]

# Create a dictionary and a corpus for LDA
dictionary = Dictionary(processed_data1)
corpus = [dictionary.doc2bow(doc) for doc in processed_data1]

# Train the LDA model
num_topics = 3  # Number of topics
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=5)

# Extract the dominant topic for each document
lda_topics = []
for bow in corpus:
    topic_distribution = lda_model.get_document_topics(bow, minimum_probability=0.0)
    dominant_topic = max(topic_distribution, key=lambda x: x[1])[0]  # Get the topic with max probability
    lda_topics.append(dominant_topic)

# Add the dominant topic as a feature in the DataFrame
df['lda_topic'] = lda_topics




In [ ]:
# Combine the original text with the LDA topic information
df['text_with_topic'] = df['combined_text'] + ' topic_' + df['lda_topic'].astype(str)


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize the combined text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text_with_topic'])
sequences = tokenizer.texts_to_sequences(df['text_with_topic'])

# Pad sequences to ensure uniform input size
max_len = 100  # Maximum length of input sequences
X = pad_sequences(sequences, padding='post', maxlen=max_len)


In [ ]:
import numpy as np
from fasttext import load_model

# Load your trained FastText model
model = load_model("supervised_fasttext_model.bin")

# Create embedding matrix
embedding_dim = 300  # FastText embedding dimension
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))

for word, index in tokenizer.word_index.items():
    try:
        embedding_matrix[index] = model.get_word_vector(word)
    except KeyError:
        # If word not in FastText vocabulary, leave it as zeros
        pass


In [ ]:
from sklearn.model_selection import train_test_split

# Prepare labels (binary classification: fake = 1, real = 0)
y = df['label'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
import tensorflow as tf
from keras import backend as K
from keras.layers import Layer

class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        # Learnable weight for attention mechanism
        self.W = self.add_weight(shape=(input_shape[-1], 1), initializer="normal", trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, inputs):
        # Ensure inputs are either a single tensor or a list of [query, value]
        if isinstance(inputs, list) and len(inputs) == 2:
            query, value = inputs
        else:
            query, value = inputs, inputs

        # Calculate attention scores using the query and value
        attention_scores = tf.matmul(value, self.W)  # Shape: (batch_size, seq_length, 1)
        attention_weights = tf.nn.softmax(attention_scores, axis=1)  # Shape: (batch_size, seq_length, 1)

        # Apply attention weights to the value tensor
        weighted_value = value * attention_weights  # Shape: (batch_size, seq_length, features)

        # Reduce across the sequence dimension to obtain the context vector
        context_vector = tf.reduce_sum(weighted_value, axis=1)  # Shape: (batch_size, features)
        return context_vector


In [ ]:
from tensorflow.keras.layers import Layer, InputSpec
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Create embedding matrix (300 dimensions for FastText)
embedding_dim = 300
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))
for word, index in tokenizer.word_index.items():
    if word in model:  # Assuming `model` is your trained FastText embedding
        embedding_matrix[index] = model.get_word_vector(word)

# Prepare labels
y = df['label'].values  # Replace 'label' with your label column

# One-hot encode the labels
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y.reshape(-1, 1))  # Adjusts for categorical labels

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:


# CNN-LSTM Model with Attention
cnn_lstm_model = Sequential()

# Embedding layer
cnn_lstm_model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                             output_dim=embedding_dim,
                             weights=[embedding_matrix],
                             input_length=max_len,
                             trainable=False))

# Conv1D + MaxPooling layers
cnn_lstm_model.add(Conv1D(filters=64, kernel_size=4, activation='relu', kernel_regularizer=l2(0.01)))
cnn_lstm_model.add(MaxPooling1D(pool_size=2))
cnn_lstm_model.add(Conv1D(filters=64, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01)))
cnn_lstm_model.add(MaxPooling1D(pool_size=2))

# LSTM Layers
cnn_lstm_model.add(LSTM(units=50, return_sequences=True, kernel_regularizer=l2(0.01)))
cnn_lstm_model.add(LSTM(units=30, return_sequences=True, kernel_regularizer=l2(0.01)))  # Return sequences for Attention

# Attention Layer
cnn_lstm_model.add(Attention())

# Flatten layer
cnn_lstm_model.add(Flatten())

# Fully connected Dense layers with Dropout
cnn_lstm_model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
cnn_lstm_model.add(Dropout(0.5))
cnn_lstm_model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
cnn_lstm_model.add(Dropout(0.5))

# Output layer
cnn_lstm_model.add(Dense(y.shape[1], activation='softmax'))  # Multi-class classification

# Compile the model
cnn_lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])




In [ ]:
# Train the model
cnn_lstm_model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_accuracy = cnn_lstm_model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy}')

# Generate predictions
y_pred = cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)


In [ ]:
from sklearn.metrics import classification_report

# Convert numeric class labels to string labels
target_names = [str(cls) for cls in encoder.categories_[0]]

# Generate the classification report
report = classification_report(y_true_classes, y_pred_classes, target_names=target_names)
print("\nClassification Report:\n")
print(report)


In [ ]:
# Predict on new examples
new_examples = [
    "court order nigeria pay million war victim abuja reuters court monday ordered nigeria pay billion naira million damage victim civil war failure fully demine clear land weaponry end hostility ruling call government pay billion naira directly war victim state put billion naira toward demining construction school court church mosque affected area judge economic community west african state court justice ruled remain large quantity live bomb deprived community farmland since civil war ended sovereign state respect court ruling framework making binding office president muhammadu buhari immediately comment ruling million people died civil war shortlived republic biafra naira",
    "america give grand piano horse wednesday november lucas wilde america give grand piano horse america given grand piano horse expecting quality tune im particularly looking forward beethoven ninth beamed horse supporter piano enthusiast jay cooper horse never given piano frankly establishment wouldnt allow last change come america change better lot doubter doubter soon silenced graceful note chopin mozart maybe even little richard horse dobbin williams said im really sure whats expected im horse absolutely qualified play piano mean look hoof way general cant even sit chair properly earth anyone think good idea cooper grinned weve made piano great democrat elizabeth king said wanted get pianist lowtomedium standard piano wouldnt thumped anything exciting would perfectly reasonable background music people spoken people wanted horse god bless america get best newsthump story mailbox every friday free currently witterings add"
]
new_sequences = tokenizer.texts_to_sequences(new_examples)
new_padded = pad_sequences(new_sequences, padding='post', maxlen=max_len)

# Get predictions
predictions = cnn_lstm_model.predict(new_padded)

# Interpret predictions
for example, prediction in zip(new_examples, predictions):
    print(f"Text: {example}")
    print(f"Prediction: {'Real News' if prediction[0] < 0.5 else 'Fake News'}")


In [ ]:
pip install lime


In [ ]:
# Predict on new examples
new_examples = [
    "court order nigeria pay million war victim abuja reuters court monday ordered nigeria pay billion naira million damage victim civil war failure fully demine clear land weaponry end hostility ruling call government pay billion naira directly war victim state put billion naira toward demining construction school court church mosque affected area judge economic community west african state court justice ruled remain large quantity live bomb deprived community farmland since civil war ended sovereign state respect court ruling framework making binding office president muhammadu buhari immediately comment ruling million people died civil war shortlived republic biafra naira",
    "america give grand piano horse wednesday november lucas wilde america give grand piano horse america given grand piano horse expecting quality tune im particularly looking forward beethoven ninth beamed horse supporter piano enthusiast jay cooper horse never given piano frankly establishment wouldnt allow last change come america change better lot doubter doubter soon silenced graceful note chopin mozart maybe even little richard horse dobbin williams said im really sure whats expected im horse absolutely qualified play piano mean look hoof way general cant even sit chair properly earth anyone think good idea cooper grinned weve made piano great democrat elizabeth king said wanted get pianist lowtomedium standard piano wouldnt thumped anything exciting would perfectly reasonable background music people spoken people wanted horse god bless america get best newsthump story mailbox every friday free currently witterings add"
]
new_sequences = tokenizer.texts_to_sequences(new_examples)
new_padded = pad_sequences(new_sequences, padding='post', maxlen=max_len)

# Get predictions
predictions = cnn_lstm_model.predict(new_padded)

# Interpret predictions
for example, prediction in zip(new_examples, predictions):
    print(f"Text: {example}")
    print(f"Prediction: {'Real News' if prediction[0] < 0.5 else 'Fake News'}")


import numpy as np
import lime
from lime.lime_text import LimeTextExplainer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Create a function to predict using the CNN-LSTM model
def predict_proba(texts):
    # Tokenize and pad the texts for prediction
    sequences = tokenizer.texts_to_sequences(texts)
    X = pad_sequences(sequences, padding='post', maxlen=max_len)
    
    # Get the model's prediction probabilities (returns probability of "Fake News")
    proba = cnn_lstm_model.predict(X)
    
    # Return probabilities for both classes (Real News: 1 - proba, Fake News: proba)
    return np.column_stack((1 - proba, proba))

# Instantiate the LIME Text Explainer
explainer = LimeTextExplainer(class_names=['Real News', 'Fake News'])

# Select a sample text for explanation (from test set or any other example)
# sample_text = ["court order nigeria pay million war victim abuja reuters court monday ordered nigeria pay billion naira million damage victim civil war failure fully demine clear land weaponry end hostility ruling call government pay billion naira directly war victim state put billion naira toward demining construction school court church mosque affected area judge economic community west african state court justice ruled remain large quantity live bomb deprived community farmland since civil war ended sovereign state respect court ruling framework making binding office president muhammadu buhari immediately comment ruling million people died civil war shortlived republic biafra naira"]

# Explain the prediction for the sample text
explanation = explainer.explain_instance(new_examples[0], predict_proba, num_features=10)
# explanation = explainer.explain_instance(new_examples[1], predict_proba, num_features=10)

# Visualize the explanation
explanation.show_in_notebook()

# Alternatively, to print the explanation:
print("Explanation:")
for feature, weight in explanation.as_list():
    print(f"{feature}: {weight:.4f}")


In [ ]:
import matplotlib.pyplot as plt

# Assuming you have history object from model.fit()
history = cnn_lstm_model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

# Plotting loss and accuracy curves
plt.figure(figsize=(12, 6))

# Plot training & validation accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot training & validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Get predictions
y_pred = cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert predictions to class labels
y_true_classes = np.argmax(y_test, axis=1)

# Compute confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)

# Plotting the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=encoder.categories_[0], yticklabels=encoder.categories_[0])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc

# Assuming binary classification
y_pred_prob = cnn_lstm_model.predict(X_test)[:, 1]  # Get probabilities for the positive class

fpr, tpr, thresholds = roc_curve(y_true_classes, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()


In [ ]:
from sklearn.metrics import classification_report

# Assuming the classes are 'Real News' and 'Fake News'
target_names = ['Real News', 'Fake News']  # Replace with your actual class names

# Print classification report
report = classification_report(y_true_classes, y_pred_classes, target_names=target_names)
print(report)


In [ ]:
from gensim.models.coherencemodel import CoherenceModel
coherence_model = CoherenceModel(model=lda_model, texts=processed_data1, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()

    # Calculate perplexity score
perplexity_score = lda_model.log_perplexity(corpus)

print("coherence_score ", coherence_score )
print("perplexity_score",perplexity_score)

